**Melbrn Data estimator mae: 13,289**

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import numpy as np
import sys

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#helpers
USE_LOG = 0
def log_data(data):
    if USE_LOG == 1:
        return np.log(data)
    else:
        return data

def delog_data(data):
    if USE_LOG == 1:
        return np.exp(data)
    else:
        return data
    
def substitute_row_log_value(data, name_of_colubn):
    data[name_of_colubn] = np.log(data[name_of_colubn])

In [ ]:
data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

#print ("data columns:", len(data.columns),len(test_data.columns))
price = data['SalePrice']

data.dropna(axis=0, subset=['SalePrice'], inplace=True)

test_data['SalePrice'] = pd.Series(np.random.randn(len(data)-1), index=test_data.index)
test_data.dropna(axis=0, subset=['SalePrice'], inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

#corelation
corrVal = data.corr()
print (corrVal['SalePrice'].sort_values(ascending=True)[0:10])


bsmCatCol = ['BsmtCond', 'BsmtQual','BsmtFinType1','BsmtFinType2','BsmtExposure']
bsmNumCol = ['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF']


bsm = data[bsmCatCol].copy()
tbsm = test_data[bsmCatCol].copy()

numbsm = data[bsmNumCol].copy()
tnumbsm = test_data[bsmNumCol].copy()

print (bsm.head(5))

for cl in bsm.columns:
    bsm[cl] = bsm[cl].fillna('None')
    tbsm[cl] = tbsm[cl].fillna('None')


cond_encoder = LabelEncoder()
for cl in bsm.columns:
    bsm[cl] = cond_encoder.fit_transform(bsm[cl])
    tbsm[cl] = cond_encoder.fit_transform(tbsm[cl])

cond_encoder = LabelEncoder()
for cl in numbsm.columns:
    numbsm[cl] = cond_encoder.fit_transform(numbsm[cl])
    tnumbsm[cl] = cond_encoder.fit_transform(tnumbsm[cl])
    
    
print(bsm.head(5))


In [ ]:
#################################
# Features from FT
#################################

data['AllSF'] = data['TotalBsmtSF']+ data['1stFlrSF'] + data['2ndFlrSF']
test_data['AllSF'] = test_data['TotalBsmtSF']+ test_data['1stFlrSF'] + test_data['2ndFlrSF']

data['Baths']=data['BsmtFullBath'] + data['FullBath'] + data['BsmtHalfBath'] + data['HalfBath']
test_data['Baths']=test_data['BsmtFullBath'] + test_data['FullBath'] + test_data['BsmtHalfBath'] + test_data['HalfBath']

data['YrBltRemodAndGarage']=data['YearBuilt']+data['YearRemodAdd'] +data['GarageYrBlt']
test_data['YrBltRemodAndGarage']=test_data['YearBuilt']+test_data['YearRemodAdd'] +test_data['GarageYrBlt']


data['Porch']=data['ScreenPorch']+ data['OpenPorchSF']
test_data['Porch']=test_data['ScreenPorch'] +  test_data['OpenPorchSF']


data['YrConst']=data['YearBuilt'] + data['GarageYrBlt']
test_data['YrConst']=test_data['YearBuilt']+test_data['GarageYrBlt']

data['GrLivArX']=bsm['BsmtCond']*numbsm['BsmtFinSF1'] + bsm['BsmtQual']*numbsm['BsmtFinSF1']
test_data['GrLivArX']=tbsm['BsmtCond']*tnumbsm['BsmtFinSF1'] + tbsm['BsmtQual']*tnumbsm['BsmtFinSF1']


y = log_data(data.SalePrice)

#====================
cols_with_missing = [col for col in data.columns if data[col].isnull().any()]

low_cardinality_cols = [cname for cname in data.columns if 
#                        data[cname].nunique() < 15 and
                        data[cname].dtype == "object"]
numeric_cols = [cname for cname in data.columns if 
                                data[cname].dtype in ['int64', 'float64']]

my_cols = low_cardinality_cols + numeric_cols
train_predictors = data[my_cols]
test_predictors = test_data[my_cols]

print ("after dropping and adding txt:", len(train_predictors.columns), len(test_predictors.columns))

one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)
one_hot_encoded_test_predictors = pd.get_dummies(test_predictors)

final_train, final_test = one_hot_encoded_training_predictors.align(one_hot_encoded_test_predictors,
                                                                    join='left', 
                                                                    axis=1)
#use it
X = final_train
tX = final_test

X = final_train.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])
tX = final_test.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])



In [ ]:
import seaborn as sns
#ax = sns.boxplot(x=X["MSSubClass"])
USE_PLOT = 1

if USE_PLOT==1:
    fig = plt.figure(figsize=(17, 235))

    for i,c in zip(range(0,50),final_train.columns):
        fig.add_subplot(150,2, i+1)
        sns.boxplot(y=final_train.iloc[:,i])
        sns.regplot(final_train[c], price)
    plt.tight_layout()
    plt.show()

In [ ]:

#==============
# split dataset to train and test datasets
train_X, test_X, train_y, test_y = train_test_split(X.values, y.values, test_size=0.17,random_state=23)

# most_frequent, mean,median
# use inputter to replace missing values
my_imputer = SimpleImputer(strategy='median')
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

tX = my_imputer.transform(tX)

print ("columns: ", train_X.shape)

In [ ]:
# feature elimination
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel

alldata = pd.concat([data.copy(),test_data.copy()])
ay = alldata.SalePrice

falldata = alldata.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])


falldata.shape
print(falldata.columns)

In [ ]:
#clean data
cfalldata = falldata.copy()

for cl in cfalldata.columns:
    print(cl,cfalldata[cl].isna().sum())

In [ ]:
#handele nans
cfalldata['LotFrontage'] = cfalldata['LotFrontage'].fillna(0)
cfalldata['MasVnrArea'] = cfalldata['MasVnrArea'].fillna(0)
cfalldata['GarageYrBlt'] = cfalldata['GarageYrBlt'].fillna(cfalldata['GarageYrBlt'].median())
cfalldata['YrBltRemodAndGarage'] = cfalldata['YrBltRemodAndGarage'].fillna(cfalldata['YrBltRemodAndGarage'].median())
cfalldata['YrConst'] = cfalldata['YrConst'].fillna(cfalldata['YrConst'].median())


for cl in cfalldata.columns:
    if cfalldata[cl].isna().sum() >0:
        cfalldata[cl] = cfalldata[cl].fillna(cfalldata['YrConst'].median())


print(cfalldata.shape)

In [ ]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))

falldata = cfalldata.copy()
print (falldata.shape)

In [ ]:
#handle categorical values
catalldata = alldata.select_dtypes(include=['object'])

print (catalldata.columns,catalldata.shape )

In [ ]:
for cl in catalldata.columns:
    print(cl,catalldata[cl].isna().sum())

In [ ]:
#handle missing values
for cl in catalldata.columns:
    print(cl,catalldata[cl].unique())


In [ ]:
#transform to umeric
for cl in catalldata.columns:
    catalldata[cl] = catalldata[cl].fillna('None')

cond_encoder = LabelEncoder()
for cl in catalldata.columns:
    catalldata[cl] = cond_encoder.fit_transform(catalldata[cl])


In [ ]:
for cl in catalldata.columns:
    print(cl,catalldata[cl].unique())
    
print(catalldata.shape)

In [ ]:
print(falldata.shape, catalldata.shape, type(falldata),type(catalldata))

fcatalldata = pd.concat([falldata, catalldata],axis=1)

print(fcatalldata.shape)

In [ ]:
# craft new data

cdata = fcatalldata.copy()

cdata['AllSF'] = cdata['TotalBsmtSF']+ cdata['1stFlrSF'] + cdata['2ndFlrSF']


In [ ]:
iddata = cdata['Id']+1460
cdata = cdata.drop(columns=['Id'])

xcdata = cdata[:1460]
xval = cdata[:1459]

xay = ay[:1460]

print(xcdata.shape,xcdata.shape, ay.shape)

In [ ]:
#split data
tr_X, te_X, tr_y, te_y = train_test_split(xcdata.values, xay.values, test_size=0.17,random_state=23)

In [ ]:
from sklearn import ensemble
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib inline

params = {'n_estimators': 1000, 
          'max_depth': 4, 
          'min_samples_split': 2,
          'learning_rate': 0.05, 
          'loss': 'ls',
          'alpha':0.9,
          'random_state':0,
          'subsample': 0.8,
          'tol':0.0001, 
          'min_samples_leaf':3,
          'validation_fraction':0.1,
          'criterion':'friedman_mse',
          'max_features':0.3
         }

n_jobs=5

clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(tr_X, tr_y)

sc = clf.score(tr_X,tr_y)
mse = mean_squared_error(te_y, clf.predict(te_X))

print("-------------------")
print("secound approach")
print("MSE: %.4f" % mse)

val_predictionsx = clf.predict(te_X)
val_mae = mean_absolute_error(te_y,val_predictionsx)
print("mae: {:,.0f}".format(val_mae))
print("score:", sc)
print ('min:',int(np.min(val_predictionsx)),'max:',int(np.max(val_predictionsx)) )
print("-------------------")

In [ ]:
print(iddata.shape,xval.shape)


In [ ]:
USE_OUT_NEW = 0

if USE_OUT_NEW == 1:
    val_predictionsxx = clf.predict(xval)

    output = pd.DataFrame({'Id': iddata[:1459], 'SalePrice': val_predictionsxx})
    output.to_csv('submission.csv', index=False)

In [ ]:
from sklearn import ensemble
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib inline

for nr in range(0,1):
    params = {'n_estimators': 10000, 
              'max_depth': 4, 
              'min_samples_split': 2,
              'learning_rate': 0.05, 
              'loss': 'ls',
              'alpha':0.9,
              'random_state':36,
              'subsample': 0.8,
              'tol':0.0001, 
              'min_samples_leaf':3,
              'validation_fraction':0.1,
              'criterion':'friedman_mse',
              'max_features':0.3
             }

    n_jobs=5

    clf = ensemble.GradientBoostingRegressor(**params)

    clf.fit(train_X, train_y)

    sc = clf.score(train_X,train_y)
    mse = mean_squared_error(test_y, clf.predict(test_X))

    print("-------------------")
    print("first approach")
    print("MSE: %.4f" % mse)

    val_predictions = clf.predict(test_X)
    val_mae = mean_absolute_error(delog_data(test_y),delog_data(val_predictions))
    print("mae: {:,.0f}".format(val_mae))
    print("score:", sc)
    print ('min:',int(np.min(val_predictions)),'max:',int(np.max(val_predictions)) )
    print("-------------------")


In [ ]:
USE_OUT = 1

if USE_OUT == 1:
    val_predictions = clf.predict(tX)

    for val in range(0,5):
        print("Id:", test_data.Id[val],"SalePrice:", y[val], "SalePricePred:",val_predictions[val])

    output = pd.DataFrame({'Id': test_data.Id,'SalePrice': delog_data(val_predictions)})
    output.to_csv('submission.csv', index=False)
